In [1]:
import numpy as np
import math
import pandas as pd
from logregress import log_regress
from logregress import lin_regress
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

In [2]:
megaframe = pd.read_csv('../datasets/MEGAFRAME_CLEANEDV2.csv')
megaframe.head()

,TIME_PERIOD,Reference area,REF_AREA,Trade union density,Combined corporate income tax rate,Education spending,Health spending,Housing spending,Community development spending,IRLT,UNEMP,"Population, total",GDP per capita (current US$),"Inflation, consumer prices (annual %)",Gini index,Region
0,2001,Sweden,SWE,78.699997,28.0,0.073432,0.063153,0.005750,0.002563,5.107500,5.825000,8895960.0,27259.480674,2.405958,26.5,Europe and Central Asia
1,2002,Sweden,SWE,78.000000,28.0,0.074421,0.066268,0.005462,0.002885,5.303333,5.950000,8924958.0,29957.938765,2.158482,26.2,Europe and Central Asia
2,2003,Sweden,SWE,77.199997,28.0,0.073959,0.068195,0.005385,0.002733,4.638333,6.566667,8958229.0,37292.421434,1.925655,25.3,Europe and Central Asia
3,2004,Sweden,SWE,76.400002,28.0,0.072734,0.066088,0.005046,0.002877,4.425000,7.375000,8993531.0,42758.201550,0.373660,26.1,Europe and Central Asia
4,2005,Sweden,SWE,75.699997,28.0,0.072877,0.066296,0.003736,0.002917,3.382500,7.783333,9029572.0,43378.615212,0.453171,26.8,Europe and Central Asia


In [3]:
master_df = pd.get_dummies(megaframe, columns=['Region'], dtype=int)
master_df

,TIME_PERIOD,Reference area,REF_AREA,Trade union density,Combined corporate income tax rate,Education spending,Health spending,Housing spending,Community development spending,IRLT,UNEMP,"Population, total",GDP per capita (current US$),"Inflation, consumer prices (annual %)",Gini index,Region_East Asia and Pacific,Region_Europe and Central Asia,Region_Latin America and Caribbean,Region_Middle East and North Africa
0,2001,Sweden,SWE,78.699997,28.0,0.073432,0.063153,0.005750,0.002563,5.107500,5.825000,8895960.0,27259.480674,2.405958,26.5,0,1,0,0
1,2002,Sweden,SWE,78.000000,28.0,0.074421,0.066268,0.005462,0.002885,5.303333,5.950000,8924958.0,29957.938765,2.158482,26.2,0,1,0,0
2,2003,Sweden,SWE,77.199997,28.0,0.073959,0.068195,0.005385,0.002733,4.638333,6.566667,8958229.0,37292.421434,1.925655,25.3,0,1,0,0
3,2004,Sweden,SWE,76.400002,28.0,0.072734,0.066088,0.005046,0.002877,4.425000,7.375000,8993531.0,42758.201550,0.373660,26.1,0,1,0,0
4,2005,Sweden,SWE,75.699997,28.0,0.072877,0.066296,0.003736,0.002917,3.382500,7.783333,9029572.0,43378.615212,0.453171,26.8,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,2015,Finland,FIN,67.500000,20.0,0.063628,0.072967,0.004367,0.002398,0.723333,9.458333,5479531.0,42560.345677,-0.207929,27.1,0,1,0,0
396,2016,Finland,FIN,65.699997,20.0,0.062434,0.072201,0.005007,0.002295,0.365000,8.916667,5495303.0,43451.256244,0.356685,27.1,0,1,0,0
397,2017,Finland,FIN,62.900002,20.0,0.058111,0.070630,0.005621,0.002109,0.546667,8.766667,5508214.0,46085.017474,0.754015,27.4,0,1,0,0
398,2018,Finland,FIN,60.000000,20.0,0.057562,0.071478,0.006421,0.002053,0.661667,7.433333,5515525.0,49654.249704,1.083821,27.3,0,1,0,0


In [33]:
population_log = False

X_df = pd.get_dummies(megaframe, columns=['Region'], dtype=int)
if population_log:
    target_idx = X_df.columns.get_loc('Population, total')
    log_pop = X_df['Population, total'].apply(lambda x: math.log(x,10))
    X_df.insert(target_idx + 1, 'Log Population', log_pop)
    X_df.drop(labels=['Population, total'], axis=1)
X_df = X_df.drop(labels=['TIME_PERIOD', 'Reference area', 'REF_AREA', 'Gini index'], axis=1)


X = X_df.to_numpy()

# Assume X is your full NumPy array
num_exclude = 4

# Split the array
X_main = X[:, :-num_exclude]   # Columns to standardize
X_last = X[:, -num_exclude:]   # Columns to leave untouched

# Standardize the first part
mean = X_main.mean(axis=0)
std = X_main.std(axis=0)
std[std == 0] = 1  # Avoid division by zero
X_main_standardized = (X_main - mean) / std

# Concatenate back together
X = np.hstack([X_main_standardized, X_last])

y = master_df['Gini index'].apply(lambda x: x/100).to_numpy()
X.shape

w = np.ones(len(X[0]))
w

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [ ]:
#logistic regression
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=3)

cval_weights = log_regress(X,y,w,alpha=.1,max_iter=10000)
predictions = []
for x in X_test:
    dot_prod = np.dot(x, cval_weights)
    y_hat = 1 / (1 + np.exp(-dot_prod))
    predictions.append(y_hat)

column_names = X_df.columns
log_cval_results = {
    'ypreds': predictions,
    'mse': mean_squared_error(y_test, predictions),
    'r2': r2_score(y_test, predictions),
    'resids': [y_test[i] - predictions[i] for i in range(len(y_test))],
    'coefficients': {column_names[i]: cval_weights[i] for i in range(len(cval_weights))}
}

In [45]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=3)

cval_weights = lin_regress(X,y)
predictions = []
for x in X_test:
    y_hat = np.dot(x,cval_weights)
    predictions.append(y_hat)

column_names = X_df.columns
lin_cval_results = {
    'ypreds': predictions,
    'mse': mean_squared_error(y_test, predictions),
    'r2': r2_score(y_test, predictions),
    'resids': [y_test[i] - predictions[i] for i in range(len(y_test))],
    'coefficients': {column_names[i]: cval_weights[i] for i in range(len(cval_weights))}
}

In [46]:
print('linear r2:', lin_cval_results['r2'])
print('logistic r2:', log_cval_results['r2'])
lin_cval_results['coefficients']

linear r2: 0.6907353425294364
logistic r2: 0.4680970373469502


{'Trade union density': -0.01590215205562584,
 'Combined corporate income tax rate': -0.002657754577600327,
 'Education spending': 0.010917773995207656,
 'Health spending': -0.009051435765733522,
 'Housing spending': 0.002993422243756192,
 'Community development spending': -0.0004333729544671649,
 'IRLT': 0.0017525949705529348,
 'UNEMP': 0.011690398456695571,
 'Population, total': 0.013550247928680164,
 'GDP per capita (current US$)': 0.009428127863406764,
 'Inflation, consumer prices (annual %)': 0.004860250345119562,
 'Region_East Asia and Pacific': 0.32542172572703415,
 'Region_Europe and Central Asia': 0.3068476555189667,
 'Region_Latin America and Caribbean': 0.4962985345068884,
 'Region_Middle East and North Africa': 0.38170749082472405}

In [47]:
import plotly.express as px

px.scatter(x=np.array(predictions), y=y_test)

In [48]:
px.scatter(x = range(len(lin_cval_results['resids'])), y = lin_cval_results['resids'])

In [49]:
for i in range(len(X_df.columns)):
    colname = X_df.columns[i]
    fig = px.scatter(x = X_test[:, i], y = lin_cval_results['resids'], title=f'Residual plot of {colname}', labels={'x': colname, 'y': 'Residuals'})
    fig.show()